In [18]:
!pip install pycountry
!pip install langdetect
!pip install transliterate

You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/droman/Documents/stuff/dl_course/venv/bin/python -m pip install --upgrade pip' command.


In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

%pylab inline
plt.style.use("bmh")
import pandas as pd

from tqdm import tqdm
tqdm.pandas()
from functools import reduce

import pycountry
import re

import langdetect
from transliterate import translit, get_available_language_codes, slugify

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to /home/droman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/droman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/droman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/droman/Documents/stuff/dl_course/venv/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [22]:
train = pd.read_csv("./train_converted.csv")[:100]

In [23]:
def get_popular_words(data):
    documents = pd.concat([data['name_1'], data['name_2']]).values.astype('U')
    count_vectorizer = CountVectorizer(max_features=1000) # memory limit: ngram_range=(2, 5),
    values = count_vectorizer.fit_transform(documents)
    frequency = values.toarray().sum(axis=0)

    feature_names = count_vectorizer.get_feature_names()
    count_data = pd.DataFrame({'feature_names': feature_names, 'frequency': frequency})


    tfidf_vectorizer = TfidfVectorizer(max_features=1000) # memory limit: ngram_range=(2, 5),
    values = tfidf_vectorizer.fit_transform(documents)
    frequency = values.toarray().sum(axis=0)

    feature_names = tfidf_vectorizer.get_feature_names()
    tf_idf_data = pd.DataFrame({'feature_names': feature_names, 'frequency': frequency})


    tf_idf_data = tf_idf_data.sort_values(by=['frequency'], ascending=False)
    count_data = count_data.sort_values(by=['frequency'], ascending=False)

    perc = 0.2
    size = int(len(count_data) * perc)

    count_stop = count_data['feature_names'].iloc[:size].values
    tf_idf_stop = tf_idf_data['feature_names'].iloc[:size].values

    popular_words = set(count_stop) | set(tf_idf_stop)
    return popular_words

def transliterate_text(text):
  t = slugify(text)
  return t if t is not None else text

def clear_data(data):
    # cities = pd.read_csv("../cities.csv", header=None, encoding='iso-8859-1')
    stop_cities = []
    # companies = pd.read_csv("../2020-11-19_elf-code-list-v1.3.csv")
    # duplicate_companies = []
    # for i, (name_1, name_2) in companies.iterrows():
    #     if isinstance(name_1, str) and isinstance(name_2, str):
    #         if name_1 != name_2:
    #             duplicate_companies.append((name_1, name_2))

    data['name_1'] = data['name_1'].apply(transliterate_text)
    data['name_2'] = data['name_2'].apply(transliterate_text)

    countries = [country.name.lower() for country in pycountry.countries]

    data["name_1"] = data["name_1"].str.lower()
    data["name_2"] = data["name_2"].str.lower()

    # REMOVE COMPANIES LABELS
    legal_entities = ["ltd.", "co.", "inc.", "b.v.", "s.c.r.l.", "gmbh", "pvt."]

    for entity in tqdm(legal_entities):
        data.replace(re.compile(f"\s+{entity}\s*"), "", inplace=True)

    # REMOVE DATA IN CORNERS
    data.replace(re.compile(r"\s+\(.*\)"), "", inplace=True)
    data.replace(re.compile(r"[^\w\s]"), "", inplace=True)

    # REMOVE COUNTRY WORDS
    for country in tqdm(countries):
        data.replace(re.compile(country), "", inplace=True)

    stop_words = stopwords.words("english")

    data['name_1'] = data['name_1'].apply(lambda sentence: ' '.join([w for w in sentence.rstrip().split() if not w in stop_words]))
    data['name_2'] = data['name_2'].apply(lambda sentence: ' '.join([w for w in sentence.rstrip().split() if not w in stop_words]))

    # LEMMATIZE
    # приводит к нормальной форме
    lemmatizer = WordNetLemmatizer() #  lemmatizer.lemmatize(word)
    # отрезает окончание
    stemmer = PorterStemmer() # stemmer.stem(word)

    data['name_1'] = data['name_1'].apply(lambda sentence: ' '.join([lemmatizer.lemmatize(w) for w in sentence.rstrip().split()]))
    data['name_2'] = data['name_2'].apply(lambda sentence: ' '.join([lemmatizer.lemmatize(w) for w in sentence.rstrip().split()]))

    data.fillna('', inplace=True)
    popular_words = get_popular_words(data)

    data['name_1'] = data['name_1'].apply(lambda s: ' '.join([w for w in s.rstrip().split() if not w in popular_words]))
    data['name_2'] = data['name_2'].apply(lambda s: ' '.join([w for w in s.rstrip().split() if not w in popular_words]))

    data = data.drop_duplicates()
    data = data.dropna()
    return data

In [24]:
data = clear_data(train)
data.to_csv("test_clear_data.csv")

100%|██████████| 249/249 [00:00<00:00, 1671.43it/s]


In [ ]:
def get_extra_relations(data):
    data = data[data.is_duplicate==1]
    data = data[['name_1','name_2']]

    def get_same_companies_graph(companies_dataframe):
        result = [] # [[[a,b,c], {a: [b, c], b: [c]}, 3], [[d,e], {d: [e]}], 2]
        for i, pair in tqdm(companies_dataframe.iterrows()):
            first = pair['name_1']
            second = pair['name_2']
            if first != second:
                flag = True

                for list_same_companies in result:
                    if first in list_same_companies[0]:
                        if second not in list_same_companies[0]:
                            list_same_companies[0].append(second)
                            try:
                                list_same_companies[1][first].append(second)
                            except:
                                list_same_companies[1].update({first: [second]})
                            list_same_companies[2]+=1
                            flag = False
                    else:
                        if second in list_same_companies[0]:
                            if first not in list_same_companies[0]:
                                list_same_companies[0].append(first)
                                try:
                                    list_same_companies[1][second].append(first)
                                except:
                                    list_same_companies[1].update({second: [first]})
                                list_same_companies[2]+=1
                                flag = False
                if flag:
                    result.append([[first, second], {first: [second]}, 1])
        return result

    companies_graph = get_same_companies_graph(data)

    LIST_UNIQUE_COMPANIES_ID = 0
    DICT_OF_RELATIONS_ID = 1
    COUNT_OF_RELATIONS_ID = 2
    EXTRA_RELATIONS_ID = 3
    def get_extra_relations(inner_companies_graph):
        extra_relations = []
        for unique_companies in tqdm(inner_companies_graph):
            unique_companies.append([])
            companies = unique_companies[LIST_UNIQUE_COMPANIES_ID]
            company_relations = unique_companies[DICT_OF_RELATIONS_ID]
            for company in companies:
                try:
                    relations = company_relations[company]
                    excluded_companies = relations
                except:
                    excluded_companies = []
                excluded_companies.append(company)
                for extra_relation_company in companies:
                    if extra_relation_company not in excluded_companies:
                        extra_relations.append([company, extra_relation_company])
                        unique_companies[COUNT_OF_RELATIONS_ID]+=1
                        unique_companies[EXTRA_RELATIONS_ID].append([company, extra_relation_company])
        return extra_relations

    extra_relations = get_extra_relations(companies_graph)
    df = pd.DataFrame(extra_relations, columns = ['name_1','name_2'])
    df['is_duplicate'] =   1
    return df

In [ ]:
def get_abbreviation(data):
    data = data[data.is_duplicate==0]

    data = data['name_1'].append(data['name_2']).reset_index(drop=True)
    def filter_condition(sentence):
        words = sentence.split()
        return 2 < len(words) < 6 and all(len(word)>3 for word in words)

    data = list(filter(filter_condition, data.tolist()))

    def _get_abbreviation(sentence):
        return ''.join([word[0] for word in sentence.split()])
    data = [[sentence, _get_abbreviation(sentence)] for sentence in data]

    return pd.DataFrame(data, columns=['name_1', 'name_2'])